In [2]:
import pandas as pd
import numpy as np # library to handle data in a vectorized manner


!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

<h1>Step 1: Data cleaning and visualization</h1>

In [3]:
canada_df = pd.read_csv('canada.csv')

In [4]:
canada_df = canada_df.drop(['Unnamed: 0', 'Postal Code.1'], axis=1)
canada_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [6]:
# Getting latitude and longitude data for the address
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 43.6534817, -79.3839347.


In [7]:
# Folium map with neighbhorhood superimposed
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(canada_df['Latitude'], canada_df['Longitude'], canada_df['Borough'], canada_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [8]:
# FourSquare credentials
CLIENT_ID = 'Y0SB1KFG5R3HDSHQ4SRGLUIO5AJOBJPXTQIENEG5NDXYRRYD' # your Foursquare ID
CLIENT_SECRET = 'JAVZOS2UBIHBSKGGXQ12J1NCHIWIGVMB3LRRFJPFYXOU5KBU' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y0SB1KFG5R3HDSHQ4SRGLUIO5AJOBJPXTQIENEG5NDXYRRYD
CLIENT_SECRET:JAVZOS2UBIHBSKGGXQ12J1NCHIWIGVMB3LRRFJPFYXOU5KBU


<h1>Step 2: Collecting neighborhood venue data and finding most visited place<\h1>

In [9]:
canada_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
canada_df.loc[0, 'Neighborhood']

'Malvern, Rouge'

In [11]:
neighborhood_latitude = canada_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = canada_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = canada_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.8066863, -79.19435340000003.


In [12]:
# top 100 venues that are in Marble Hill within a radius of 500 meters
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=Y0SB1KFG5R3HDSHQ4SRGLUIO5AJOBJPXTQIENEG5NDXYRRYD&client_secret=JAVZOS2UBIHBSKGGXQ12J1NCHIWIGVMB3LRRFJPFYXOU5KBU&v=20180604&ll=43.8066863,-79.19435340000003&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results["response"]

{'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
 'headerLocation': 'Malvern',
 'headerFullLocation': 'Malvern, Toronto',
 'headerLocationGranularity': 'neighborhood',
 'totalResults': 1,
 'suggestedBounds': {'ne': {'lat': 43.811186304500005,
   'lng': -79.18812958073043},
  'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
 'groups': [{'type': 'Recommended Places',
   'name': 'recommended',
   'items': [{'reasons': {'count': 0,
      'items': [{'summary': 'This spot is popular',
        'type': 'general',
        'reasonName': 'globalInteractionReason'}]},
     'venue': {'id': '4bb6b9446edc76b0d771311c',
      'name': 'Wendy’s',
      'location': {'crossStreet': 'Morningside & Sheppard',
       'lat': 43.80744841934756,
       'lng': -79.19905558052072,
       'labeledLatLngs': [{'label': 'display',
         'lat': 43.80744841934756,
         'lng': -79.19905558052072}],
       'distance

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        for v in results:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name'])])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [21]:

toronto_venues = getNearbyVenues(names=canada_df['Neighborhood'],
                                   latitudes=canada_df['Latitude'],
                                   longitudes=canada_df['Longitude']
                                  )


Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [22]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


In [23]:
toronto_venues.groupby('Neighborhood').count().sort_values('Venue')  # Grouping values by Neighborhood

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Humber Summit,1,1,1,1,1,1
"Willowdale, Newtonbrook",1,1,1,1,1,1
"West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale",1,1,1,1,1,1
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",1,1,1,1,1,1
"Malvern, Rouge",1,1,1,1,1,1
...,...,...,...,...,...,...
"Toronto Dominion Centre, Design Exchange",100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [24]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


<h1>Encoding the data</h1>

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-82]] + list(toronto_onehot.columns[:-82])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,"South Steeles, Silverstone, Humbergate, Jamest...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2141,"South Steeles, Silverstone, Humbergate, Jamest...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2142,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2143,"Northwest, West Humber - Clairville",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
toronto_onehot.shape

(2145, 191)

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
toronto_grouped.shape

(97, 191)

<h3>Finding the most visited spot in each neighborhood</h3>

In [40]:
# Let's print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3          Accessories Store  0.00
4     Gluten-free Restaurant  0.00


----Alderwood, Long Branch----
                venue  freq
0        Dance Studio  0.09
1                 Gym  0.09
2         Coffee Shop  0.09
3  Athletics & Sports  0.09
4   Accessories Store  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.10
1          Coffee Shop  0.10
2       Ice Cream Shop  0.05
3  Fried Chicken Joint  0.05
4                Diner  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2          

In [41]:
# Let's put that into a pandas dataframe

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]  # exclude 0th row
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
toronto_grouped.head(3)

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
1,"Alderwood, Long Branch",Coffee Shop,Gym,Dance Studio,Athletics & Sports,Music Venue,Church,College Rec Center,College Gym,College Auditorium,College Arts Building
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Ice Cream Shop,Diner,Gas Station,Deli / Bodega,Bridal Shop,Middle Eastern Restaurant,College Gym
3,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Music Venue,Climbing Gym,College Rec Center,College Gym,College Auditorium,College Arts Building
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Juice Bar,Liquor Store,Indian Restaurant,Café,Greek Restaurant,Grocery Store
...,...,...,...,...,...,...,...,...,...,...,...
92,"Willowdale, Willowdale West",Butcher,Bank,Home Service,Grocery Store,Coffee Shop,Music Venue,Church,College Rec Center,College Gym,College Auditorium
93,Woburn,Coffee Shop,Korean Restaurant,Music Venue,Bus Station,College Rec Center,College Gym,College Auditorium,College Arts Building,Cocktail Bar,Clothing Store
94,Woodbine Heights,Bus Stop,Beer Store,Diner,Cosmetics Shop,Dance Studio,Curling Ice,Asian Restaurant,Church,College Gym,College Auditorium
95,York Mills West,Bank,Convenience Store,Music Venue,Climbing Gym,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop


<h1>Cluster the neighbhorhood<\h1>

In [54]:
#k-means to cluster the neighborhood into 5 clusters.
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0)
kmeans.fit(toronto_grouped_clustering)

valuuee = kmeans.labels_[0:]


In [57]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', valuuee)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
1,3,"Alderwood, Long Branch",Coffee Shop,Gym,Dance Studio,Athletics & Sports,Music Venue,Church,College Rec Center,College Gym,College Auditorium,College Arts Building
2,3,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Ice Cream Shop,Diner,Gas Station,Deli / Bodega,Bridal Shop,Middle Eastern Restaurant,College Gym
3,3,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Music Venue,Climbing Gym,College Rec Center,College Gym,College Auditorium,College Arts Building
4,3,"Bedford Park, Lawrence Manor East",Coffee Shop,Italian Restaurant,American Restaurant,Comfort Food Restaurant,Juice Bar,Liquor Store,Indian Restaurant,Café,Greek Restaurant,Grocery Store


In [59]:
toronto_merged = canada_df
toronto_merged = pd.merge(canada_df, neighborhoods_venues_sorted, on='Neighborhood')

In [60]:
toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Music Venue,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3,History Museum,Bar,Business Service,Butcher,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Bank,Moving Target,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Chocolate Shop,College Gym,College Auditorium
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Music Venue,Bus Station,College Rec Center,College Gym,College Auditorium,College Arts Building,Cocktail Bar,Clothing Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Fried Chicken Joint,Hakka Restaurant,Bank,Bakery,Caribbean Restaurant,Gas Station,Athletics & Sports,Cocktail Bar,College Stadium,College Rec Center


In [61]:
# Visualization

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

</h2>Examine the 5 clusters</h2>

In [63]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0]
cluster1

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
92,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724,0,Gift Shop,Music Venue,Church,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store


In [64]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1]
cluster2

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Music Venue,Bus Station,College Rec Center,College Gym,College Auditorium,College Arts Building,Cocktail Bar,Clothing Store
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Convenience Store,Music Venue,Comfort Food Restaurant,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,1,Discount Store,Hobby Shop,Bus Station,Coffee Shop,Department Store,Convenience Store,Music Venue,Chocolate Shop,College Auditorium,College Arts Building
22,M2P,North York,York Mills West,43.752758,-79.400049,1,Bank,Convenience Store,Music Venue,Climbing Gym,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
39,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,1,Convenience Store,Coffee Shop,Music Venue,Comfort Food Restaurant,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Cocktail Bar
79,M6M,York,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013,1,Convenience Store,Discount Store,Coffee Shop,Music Venue,Church,College Rec Center,College Gym,College Auditorium,College Arts Building,Cocktail Bar
93,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.643515,-79.577201,1,Café,Liquor Store,Convenience Store,Beer Store,Cosmetics Shop,Coffee Shop,Music Venue,Climbing Gym,College Gym,College Auditorium
96,M9N,York,Weston,43.706876,-79.518188,1,Convenience Store,Music Venue,Comfort Food Restaurant,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar


In [65]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2]
cluster3

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
90,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2,Baseball Field,Bus Stop,Colombian Restaurant,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar
95,M9M,North York,"Humberlea, Emery",43.724766,-79.532242,2,Baseball Field,Bus Stop,Colombian Restaurant,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar


In [66]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3]
cluster4

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3,History Museum,Bar,Business Service,Butcher,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3,Bank,Moving Target,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Chocolate Shop,College Gym,College Auditorium
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3,Fried Chicken Joint,Hakka Restaurant,Bank,Bakery,Caribbean Restaurant,Gas Station,Athletics & Sports,Cocktail Bar,College Stadium,College Rec Center
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,3,Bus Line,Bakery,Bus Station,Intersection,Metro Station,Ice Cream Shop,Chocolate Shop,College Auditorium,College Arts Building,Coffee Shop
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,3,Motel,American Restaurant,Music Venue,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M9L,North York,Humber Summit,43.756303,-79.565963,3,Music Venue,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar,Clothing Store
97,M9P,Etobicoke,Westmount,43.696319,-79.532242,3,Discount Store,Coffee Shop,Intersection,Middle Eastern Restaurant,Chinese Restaurant,Music Venue,Chocolate Shop,College Gym,College Auditorium,College Arts Building
98,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724,3,Bus Line,Music Venue,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar
99,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437,3,Grocery Store,Fried Chicken Joint,Beer Store,Fast Food Restaurant,Music Venue,Climbing Gym,College Rec Center,College Gym,College Auditorium,College Arts Building


In [67]:
cluster5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4]
cluster5

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Music Venue,Church,College Stadium,College Rec Center,College Gym,College Auditorium,College Arts Building,Coffee Shop,Cocktail Bar
